In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
from xgboost import XGBClassifier
import os


In [2]:
# ✅ 1. Load & Preprocess Data
script_dir = os.getcwd() # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_cleaned.xlsx")
df = pd.read_excel(file_path)

#visualize the data
print(df.head())
print(df.info())

                                             context  \
0  Een draft van deze visienota  is opgemaakt en ...   
1  Een draft van deze visienota  is opgemaakt en ...   
2  Een draft van deze visienota  is opgemaakt en ...   
3  Daarna  zal Fluvius  de visienota indienen bij...   
4  Wanneer ze zich daarna  opnieuw willen  inzett...   

                                            question  statistical    theme  \
0  1.Kan de VREG de opgestelde visienota van Fluv...            0  Energie   
1     a)Wat zijn de krijtlijnen  van deze visienota?            0  Energie   
2  b)Welke acties zullen op basis van deze visien...            0  Energie   
3  2.Wat is de reactie van de VREG op deze visien...            0  Energie   
4  1.Hoeveel 50-54, 55-59  en 60-plussers  zijn s...            1     Werk   

     file_name  
0  1919076.txt  
1  1919076.txt  
2  1919076.txt  
3  1919076.txt  
4  1919282.txt  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39125 entries, 0 to 39124
Data columns (to

In [3]:
# Drop unnecessary columns
if "file_name" in df.columns:
    df = df.drop(columns=["file_name"])
    df = df.drop(columns=["statistical"])


# Handle missing values
df = df.dropna(subset=["question"])
df["context"].fillna("", inplace=True)

# Clean text
def clean_text(text):
    text = re.sub(r'\n', ' ', text)  # Replace newlines with spaces
    text = re.sub(r'\b[a-z]\)\s+', ' ', text)  # Remove patterns like 'a)', 'b)', etc.
    text = re.sub(r'\b\d+\.\b', '', text)  # Remove patterns like '1.', '2.', etc.
    text = re.sub(r'\b\d+\)\b', '', text)  # Remove patterns like '1)', '2)', etc.
    text = re.sub(r'\b[i]+[.)]\b', '', text, flags=re.IGNORECASE)  # Remove patterns like 'i.', 'ii.', 'i)', etc.
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces and trim
    return text

#df["clean_text"] = (df["context"] + " " + df["question"]).apply(clean_text)
df["clean_text"] = ( df["question"]).apply(clean_text)

# Group by 'clean_text' and count unique themes
duplicates_with_diff_themes = df.groupby("clean_text")["theme"].nunique().reset_index()

# Filter rows where the number of unique themes is greater than 1
duplicates_with_diff_themes = duplicates_with_diff_themes[duplicates_with_diff_themes["theme"] > 1]

# Merge back with the original dataframe to get all rows with these 'clean_text'
filtered_df = df[df["clean_text"].isin(duplicates_with_diff_themes["clean_text"])]
# Exclude rows with these 'clean_text' from the original dataframe
df = df[~df["clean_text"].isin(duplicates_with_diff_themes["clean_text"])]


# ✅ Now: drop rare themes using original theme names
theme_counts = df["theme"].value_counts()
valid_themes = theme_counts[theme_counts >= 2].index
df = df[df["theme"].isin(valid_themes)]


C:\Users\jefva\AppData\Local\Temp\ipykernel_22256\1421595890.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["context"].fillna("", inplace=True)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

# 1. Encode labels
le = LabelEncoder()
y = le.fit_transform(df["theme"])
X = df["clean_text"]

# 2. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# 3. TF-IDF vectorization (outside pipeline for more control)
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=10000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# 4. XGBoost classifier with some tuning
clf = XGBClassifier(
    use_label_encoder=False,
    eval_metric="mlogloss",
    n_estimators=300,       #lower is faster 
    max_depth=4,           # prevent overfitting
    n_jobs=-1,             # parallel processing
    verbosity=1
)
clf.fit(X_train_vec, y_train)

# 5. Predict + decode
y_pred = clf.predict(X_test_vec)
y_test_labels = le.inverse_transform(y_test)
y_pred_labels = le.inverse_transform(y_pred)

# 6. Evaluate
print(classification_report(y_test_labels, y_pred_labels, zero_division=0))



C:\Users\jefva\anaconda3\envs\bert_env\lib\site-packages\xgboost\core.py:158: UserWarning: [22:32:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


                                                                     precision    recall  f1-score   support

                                                      Armoedebeleid       0.70      0.45      0.55        31
                                                          Begroting       0.83      0.63      0.72        68
                                         Brussel en de Vlaamse Rand       1.00      0.39      0.56        36
                                                 Buitenlands beleid       0.76      0.59      0.66       160
                                                            Cultuur       0.70      0.37      0.49       158
                                                      Dierenwelzijn       0.79      0.54      0.64       128
                                                           Economie       0.64      0.41      0.50       256
                                                            Energie       0.67      0.47      0.55       292
                  